In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SalesPrediction1026").getOrCreate()

In [2]:
holidays_events = spark.read.csv("holidays_events.csv", header=True)
items = spark.read.csv("items.csv", header=True)
oil = spark.read.csv("oil.csv", header=True)
stores = spark.read.csv("stores.csv", header=True)
test = spark.read.csv("test.csv", header=True)
train = spark.read.csv("train.csv", header=True)
transactions = spark.read.csv("transactions.csv", header=True)

In [3]:
#deal with oil dataframe in pandas with backfill, then transform it to spark dataframe
oil_pandas = pd.read_csv("oil.csv")

## Check every dataframe

In [4]:
train.show(2)

+---+----------+---------+--------+----------+-----------+
| id|      date|store_nbr|item_nbr|unit_sales|onpromotion|
+---+----------+---------+--------+----------+-----------+
|  0|2013-01-01|       25|  103665|       7.0|       null|
|  1|2013-01-01|       25|  105574|       1.0|       null|
+---+----------+---------+--------+----------+-----------+
only showing top 2 rows



In [5]:
train.count()

125497040

In [6]:
holidays_events.show(2)

+----------+-------+--------+-----------+--------------------+-----------+
|      date|   type|  locale|locale_name|         description|transferred|
+----------+-------+--------+-----------+--------------------+-----------+
|2012-03-02|Holiday|   Local|      Manta|  Fundacion de Manta|      False|
|2012-04-01|Holiday|Regional|   Cotopaxi|Provincializacion...|      False|
+----------+-------+--------+-----------+--------------------+-----------+
only showing top 2 rows



In [7]:
items.show(2)

+--------+---------+-----+----------+
|item_nbr|   family|class|perishable|
+--------+---------+-----+----------+
|   96995|GROCERY I| 1093|         0|
|   99197|GROCERY I| 1067|         0|
+--------+---------+-----+----------+
only showing top 2 rows



In [8]:
oil.show(2)

+----------+----------+
|      date|dcoilwtico|
+----------+----------+
|2013-01-01|      null|
|2013-01-02|     93.14|
+----------+----------+
only showing top 2 rows



In [9]:
stores.show(2)

+---------+-----+---------+----+-------+
|store_nbr| city|    state|type|cluster|
+---------+-----+---------+----+-------+
|        1|Quito|Pichincha|   D|     13|
|        2|Quito|Pichincha|   D|     13|
+---------+-----+---------+----+-------+
only showing top 2 rows



In [10]:
transactions.show(2)

+----------+---------+------------+
|      date|store_nbr|transactions|
+----------+---------+------------+
|2013-01-01|       25|         770|
|2013-01-02|        1|        2111|
+----------+---------+------------+
only showing top 2 rows



In [11]:
test.show(2)

+---------+----------+---------+--------+-----------+
|       id|      date|store_nbr|item_nbr|onpromotion|
+---------+----------+---------+--------+-----------+
|125497040|2017-08-16|        1|   96995|      False|
|125497041|2017-08-16|        1|   99197|      False|
+---------+----------+---------+--------+-----------+
only showing top 2 rows



In [12]:
train.head(2)

[Row(id='0', date='2013-01-01', store_nbr='25', item_nbr='103665', unit_sales='7.0', onpromotion=None),
 Row(id='1', date='2013-01-01', store_nbr='25', item_nbr='105574', unit_sales='1.0', onpromotion=None)]

## Join train and holiday_events dataframe

In [13]:
train_holiday = train.join(holidays_events, train.date == holidays_events.date, 'left_outer').drop(holidays_events.date)
train_holiday.show(2)

+---+----------+---------+--------+----------+-----------+-------+--------+-----------+------------------+-----------+
| id|      date|store_nbr|item_nbr|unit_sales|onpromotion|   type|  locale|locale_name|       description|transferred|
+---+----------+---------+--------+----------+-----------+-------+--------+-----------+------------------+-----------+
|  0|2013-01-01|       25|  103665|       7.0|       null|Holiday|National|    Ecuador|Primer dia del ano|      False|
|  1|2013-01-01|       25|  105574|       1.0|       null|Holiday|National|    Ecuador|Primer dia del ano|      False|
+---+----------+---------+--------+----------+-----------+-------+--------+-----------+------------------+-----------+
only showing top 2 rows



## Clean Oil Dataframe

In [14]:
oil_pandas.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [15]:
oil_pandas.isnull().sum()

date           0
dcoilwtico    43
dtype: int64

In [16]:
oil_pandas = oil_pandas.fillna(method='bfill')
oil_pandas = oil_pandas.fillna(method='ffill')
oil_pandas.head()

,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [17]:
oil_pandas.isnull().sum()

date          0
dcoilwtico    0
dtype: int64

## Transform pandas dataframe back to spark dataframe

In [18]:
oil_spark = spark.createDataFrame(oil_pandas)
oil_spark.show()

+----------+----------+
|      date|dcoilwtico|
+----------+----------+
|2013-01-01|     93.14|
|2013-01-02|     93.14|
|2013-01-03|     92.97|
|2013-01-04|     93.12|
|2013-01-07|      93.2|
|2013-01-08|     93.21|
|2013-01-09|     93.08|
|2013-01-10|     93.81|
|2013-01-11|      93.6|
|2013-01-14|     94.27|
|2013-01-15|     93.26|
|2013-01-16|     94.28|
|2013-01-17|     95.49|
|2013-01-18|     95.61|
|2013-01-21|     96.09|
|2013-01-22|     96.09|
|2013-01-23|     95.06|
|2013-01-24|     95.35|
|2013-01-25|     95.15|
|2013-01-28|     95.95|
+----------+----------+
only showing top 20 rows



## Join train with oil

In [19]:
train_holiday_oil = train_holiday.join(oil_spark, train.date == oil_spark.date, 'left_outer').drop(oil_spark.date)
train_holiday_oil.show()

+-------+----------+---------+--------+----------+-----------+----+------+-----------+-----------+-----------+----------+
|     id|      date|store_nbr|item_nbr|unit_sales|onpromotion|type|locale|locale_name|description|transferred|dcoilwtico|
+-------+----------+---------+--------+----------+-----------+----+------+-----------+-----------+-----------+----------+
|2924449|2013-03-14|        1|  103520|       2.0|       null|null|  null|       null|       null|       null|     93.03|
|2924450|2013-03-14|        1|  103665|       1.0|       null|null|  null|       null|       null|       null|     93.03|
|2924451|2013-03-14|        1|  105574|       2.0|       null|null|  null|       null|       null|       null|     93.03|
|2924452|2013-03-14|        1|  105575|       8.0|       null|null|  null|       null|       null|       null|     93.03|
|2924453|2013-03-14|        1|  105577|       5.0|       null|null|  null|       null|       null|       null|     93.03|
|2924454|2013-03-14|    

In [20]:
#select date = 2013-03-01 see if it's in the dataframe
train_holiday_oil.createOrReplaceTempView("train_holiday_oil_sql")
spark.sql("""
        SELECT * FROM
        train_holiday_oil_sql t
        WHERE date = '2013-01-01'
""").show()

+---+----------+---------+--------+----------+-----------+-------+--------+-----------+------------------+-----------+----------+
| id|      date|store_nbr|item_nbr|unit_sales|onpromotion|   type|  locale|locale_name|       description|transferred|dcoilwtico|
+---+----------+---------+--------+----------+-----------+-------+--------+-----------+------------------+-----------+----------+
|  0|2013-01-01|       25|  103665|       7.0|       null|Holiday|National|    Ecuador|Primer dia del ano|      False|     93.14|
|  1|2013-01-01|       25|  105574|       1.0|       null|Holiday|National|    Ecuador|Primer dia del ano|      False|     93.14|
|  2|2013-01-01|       25|  105575|       2.0|       null|Holiday|National|    Ecuador|Primer dia del ano|      False|     93.14|
|  3|2013-01-01|       25|  108079|       1.0|       null|Holiday|National|    Ecuador|Primer dia del ano|      False|     93.14|
|  4|2013-01-01|       25|  108701|       1.0|       null|Holiday|National|    Ecuador|Pri

## Join train_holiday_oil with store

### Rename store.type so it would not duplicate with holiday.type

In [21]:
stores = stores.withColumnRenamed("type", "store_type")
stores.show()

+---------+-------------+--------------------+----------+-------+
|store_nbr|         city|               state|store_type|cluster|
+---------+-------------+--------------------+----------+-------+
|        1|        Quito|           Pichincha|         D|     13|
|        2|        Quito|           Pichincha|         D|     13|
|        3|        Quito|           Pichincha|         D|      8|
|        4|        Quito|           Pichincha|         D|      9|
|        5|Santo Domingo|Santo Domingo de ...|         D|      4|
|        6|        Quito|           Pichincha|         D|     13|
|        7|        Quito|           Pichincha|         D|      8|
|        8|        Quito|           Pichincha|         D|      8|
|        9|        Quito|           Pichincha|         B|      6|
|       10|        Quito|           Pichincha|         C|     15|
|       11|      Cayambe|           Pichincha|         B|      6|
|       12|    Latacunga|            Cotopaxi|         C|     15|
|       13

In [22]:
train_holiday_oil_store = train_holiday_oil.join(stores, train_holiday_oil.store_nbr == stores.store_nbr, 'left_outer').drop(stores.store_nbr)
train_holiday_oil_store.show()

+-------+----------+---------+--------+----------+-----------+----+------+-----------+-----------+-----------+----------+-----+---------+----------+-------+
|     id|      date|store_nbr|item_nbr|unit_sales|onpromotion|type|locale|locale_name|description|transferred|dcoilwtico| city|    state|store_type|cluster|
+-------+----------+---------+--------+----------+-----------+----+------+-----------+-----------+-----------+----------+-----+---------+----------+-------+
|2924449|2013-03-14|        1|  103520|       2.0|       null|null|  null|       null|       null|       null|     93.03|Quito|Pichincha|         D|     13|
|2924450|2013-03-14|        1|  103665|       1.0|       null|null|  null|       null|       null|       null|     93.03|Quito|Pichincha|         D|     13|
|2924451|2013-03-14|        1|  105574|       2.0|       null|null|  null|       null|       null|       null|     93.03|Quito|Pichincha|         D|     13|
|2924452|2013-03-14|        1|  105575|       8.0|       n

In [23]:
transactions.show()

+----------+---------+------------+
|      date|store_nbr|transactions|
+----------+---------+------------+
|2013-01-01|       25|         770|
|2013-01-02|        1|        2111|
|2013-01-02|        2|        2358|
|2013-01-02|        3|        3487|
|2013-01-02|        4|        1922|
|2013-01-02|        5|        1903|
|2013-01-02|        6|        2143|
|2013-01-02|        7|        1874|
|2013-01-02|        8|        3250|
|2013-01-02|        9|        2940|
|2013-01-02|       10|        1293|
|2013-01-02|       11|        3547|
|2013-01-02|       12|        1362|
|2013-01-02|       13|        1102|
|2013-01-02|       14|        2002|
|2013-01-02|       15|        1622|
|2013-01-02|       16|        1167|
|2013-01-02|       17|        1580|
|2013-01-02|       18|        1635|
|2013-01-02|       19|        1369|
+----------+---------+------------+
only showing top 20 rows



## Join train_holiday_oil_store and transaction

In [24]:
#try to avoid the duplicated column name:
train_holiday_oil_store_transaction_combine = train_holiday_oil_store.join(transactions, ['date', 'store_nbr'], 'left_outer')

In [25]:
train_holiday_oil_store_transaction_combine.show()

+----------+---------+-------+--------+----------+-----------+----+------+-----------+-----------+-----------+----------+-----+---------+----------+-------+------------+
|      date|store_nbr|     id|item_nbr|unit_sales|onpromotion|type|locale|locale_name|description|transferred|dcoilwtico| city|    state|store_type|cluster|transactions|
+----------+---------+-------+--------+----------+-----------+----+------+-----------+-----------+-----------+----------+-----+---------+----------+-------+------------+
|2013-03-14|        1|2924449|  103520|       2.0|       null|null|  null|       null|       null|       null|     93.03|Quito|Pichincha|         D|     13|        1751|
|2013-03-14|        1|2924450|  103665|       1.0|       null|null|  null|       null|       null|       null|     93.03|Quito|Pichincha|         D|     13|        1751|
|2013-03-14|        1|2924451|  105574|       2.0|       null|null|  null|       null|       null|       null|     93.03|Quito|Pichincha|         D|  

## Save the file to a csv file for future use

In [27]:
train_holiday_oil_store_transaction_combine.toPandas().to_csv('train_holiday_oil_store_transaction_without_test_002.csv')

Py4JJavaError: An error occurred while calling o100.collectToPython.
: org.apache.spark.SparkException: Job 39 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:820)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:818)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:78)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:818)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:1732)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:83)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:1651)
	at org.apache.spark.SparkContext$$anonfun$stop$8.apply$mcV$sp(SparkContext.scala:1921)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1317)
	at org.apache.spark.SparkContext.stop(SparkContext.scala:1920)
	at org.apache.

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 54277)
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/anaconda/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/anaconda/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/anaconda/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/usr/local/opt/apache-spark/libexec/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/usr/local/opt/apache-spark/libexec/python/pyspark/serializers.py", line 577, in read_int
    raise EOFError
EOFError
----------------------------------------
